In [ ]:
#!/usr/bin/env python3
# noise_analysis.py
# ---------------------------------------------------------------------
#  Quantifies trajectory‑noise of simulated agents from RGB screenshots
# ---------------------------------------------------------------------
# Copyright 2025  (MIT)
# ---------------------------------------------------------------------
from __future__ import annotations

import argparse
import hashlib
import json
import logging
import math
import os
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
from typing import Dict, Tuple, List

import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from tqdm.auto import tqdm  # progress bar

# ---------------------------------------------------------------------
# LOGGING
# ---------------------------------------------------------------------
logging.basicConfig(
    format="%(levelname)s | %(asctime)s | %(message)s",
    level=logging.INFO,
)

# ---------------------------------------------------------------------
# COLOUR / TRAJECTORY UTILITIES
# ---------------------------------------------------------------------
def dominant_colours(bgr_img: np.ndarray, k: int = 4) -> np.ndarray:
    """
    K‑means in RGB space, ignoring very dark pixels.
    Returns k×3 uint8 matrix (BGR order as in OpenCV).
    """
    flat = bgr_img.reshape(-1, 3)
    flat = flat[(flat > 20).any(axis=1)]
    if len(flat) < k:                       # fallback
        return np.array([[255, 255, 255]], dtype=np.uint8)
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    km.fit(flat)
    return km.cluster_centers_.astype(np.uint8)


def mask_for_colour(hsv: np.ndarray, colour_bgr: np.ndarray,
                    tol: int = 30) -> np.ndarray:
    """
    Build an HSV range mask centred on `colour_bgr`.
    """
    colour_hsv = cv2.cvtColor(colour_bgr.reshape(1, 1, 3), cv2.COLOR_BGR2HSV)[0, 0]
    lower = np.maximum(colour_hsv - tol, 0)
    upper = np.minimum(colour_hsv + tol, 255)
    return cv2.inRange(hsv, lower, upper)


def contour_points(mask: np.ndarray) -> np.ndarray:
    """
    Returns all contour pixel coordinates (N×2: x,y).
    """
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    pts: List[Tuple[int, int]] = []
    for c in cnts:
        pts.extend(p[0] for p in c)
    return np.asarray(pts, dtype=np.int32) if pts else np.empty((0, 2), dtype=np.int32)


def poly_noise(points: np.ndarray,
               degree: int = 3) -> Tuple[float, float]:
    """
    Fit Y = f(X) polynomial (least‑squares) and return mean/std of the
    *signed* residuals (noise levels).
    """
    if len(points) < degree + 1:
        return math.nan, math.nan

    points = points[np.argsort(points[:, 0])]
    x, y = points[:, 0:1], points[:, 1]
    # remove duplicate X
    uniq, idx = np.unique(x, return_index=True)
    x, y = uniq.reshape(-1, 1), y[idx]

    poly = PolynomialFeatures(degree)
    X_poly = poly.fit_transform(x)
    model = LinearRegression().fit(X_poly, y)
    y_pred = model.predict(X_poly)
    res = y - y_pred
    return float(res.mean()), float(res.std())


# ---------------------------------------------------------------------
# IMAGE‑LEVEL PIPELINE
# ---------------------------------------------------------------------
def analyse_image(path: Path,
                  k: int,
                  poly_deg: int) -> Dict[str, float] | None:
    """
    Returns anonymised noise statistics or None if nothing detected.
    """
    img = cv2.imread(str(path))
    if img is None:
        logging.warning("Cannot read %s", path.name)
        return None

    colours = dominant_colours(img, k)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    noises = []

    for c in colours:
        pts = contour_points(mask_for_colour(hsv, c))
        if len(pts):
            mu, sd = poly_noise(pts, poly_deg)
            if not math.isnan(mu):
                noises.append((mu, sd))

    if not noises:
        return None

    mus, sds = zip(*noises)
    # histogram of residuals aggregated across colours
    hist, edges = np.histogram(np.array(mus), bins=20, density=True)

    return {
        "mean_noise": float(np.mean(mus)),
        "std_noise": float(np.mean(sds)),
        "hist": hist.tolist(),
        "bins": edges.tolist(),
    }


# ---------------------------------------------------------------------
# FILENAME → SHA‑1 DIGEST  (anonymise)
# ---------------------------------------------------------------------
def anon_key(p: Path) -> str:
    return hashlib.sha1(p.name.encode()).hexdigest()[:10]   # short hash


# ---------------------------------------------------------------------
# FOLDER‑LEVEL DRIVER
# ---------------------------------------------------------------------
def process_folder(folder: Path,
                   k: int = 4,
                   poly: int = 3,
                   jobs: int = 1) -> Dict[str, Dict[str, float]]:
    """
    Traverse folder, analyse each image, return dict keyed by anon hash.
    """
    images = [p for p in folder.iterdir() if p.suffix.lower() in {".png", ".jpg", ".jpeg"}]
    results: Dict[str, Dict[str, float]] = {}

    def _task(p: Path):
        return p, analyse_image(p, k, poly)

    iterator = images if jobs == 1 else ProcessPoolExecutor(max_workers=jobs).map(_task, images)
    for p, noise in tqdm(iterator, total=len(images), desc="analysing"):
        if noise:
            results[anon_key(p)] = noise
    return results


# ---------------------------------------------------------------------
# CLI
# ---------------------------------------------------------------------
def parse_args() -> argparse.Namespace:
    ap = argparse.ArgumentParser()
    ap.add_argument("--input", type=Path, required=True, help="folder with .png/.jpg")
    ap.add_argument("--out",   type=Path, default="noise_profile.json")
    ap.add_argument("--k",     type=int,  default=4, help="#dominant colours")
    ap.add_argument("--poly",  type=int,  default=3, help="polynomial degree")
    ap.add_argument("--jobs",  type=int,  default=1, help="parallel workers")
    return ap.parse_args()


def main():
    args = parse_args()
    if not args.input.is_dir():
        raise SystemExit(f"Input folder {args.input} does not exist")

    noise_dict = process_folder(args.input, args.k, args.poly, args.jobs)
    with open(args.out, "w") as f:
        json.dump(noise_dict, f, indent=2)
    logging.info("Wrote %d entries → %s", len(noise_dict), args.out.resolve())


if __name__ == "__main__":
    main()
